In [1]:
RRR = RealField(40);
P.<x> = PolynomialRing(ZZ);


#Generates a random polynomial of degree and norm bounded by N and no
def Rand_Pol_Fixed_Norm(N,no):
    v = vector(ZZ,N);
    for i in range(N):
        v[i] = randint(-1000*no,1000*no);
    norm = v*v;
    for i in range(N):
        v[i] = (v[i]*no)//isqrt(norm);
    f = sum(v[i]*x^i for i in range(N));
    return f;


#For fixed f,g,N,q, generates the NTRU polynomials F,G associated by f,g
def Derivation_FG(f,g,N,q):
    phi = x^N+1;
    (R_f, rho_f, iphi) = xgcd(f,phi);
    (R_g, rho_g, iphi) = xgcd(g,phi);
    assert(R_f.degree() == 0);
    assert(R_g.degree() == 0);
    (pgcd,alpha,beta) = xgcd(R_f[0],R_g[0]);
    if (pgcd != 1):
        print ("pgcd =", pgcd)
        print ("The GCD of R_f and R_g is different of 1.")
    assert(pgcd == 1);
    F = -q*beta*rho_g;
    G = q*alpha*rho_f;
    k = Compute_k(f,g,F,G,N);
    iter = 0;
    while (k!= 0):
        F = (F - k*f).quo_rem(phi)[1];
        G = (G - k*g).quo_rem(phi)[1];
        k = Compute_k(f,g,F,G,N);
    return(F,G);


#Tests if f,g can generate a NTRU lattice
def Test(f,g,N,q):
    phi = x^N+1;
    (R_f, rho_f, iphi) = xgcd(f,phi);
    (R_g, rho_g, iphi) = xgcd(g,phi);
    assert(R_f.degree() == 0);
    assert(R_g.degree() == 0);
    (pgcd,alpha,beta) = xgcd(R_f[0],R_g[0]);
    if (pgcd != 1):
        return False;
    return True;


#Returns f(1/x) mod (x^N+1)
def Reverse(f,N):
    g = sum( f[i]*(x^(N-i)) for i in [1..N-1] );
    return f[0] - g;


#Compute the polynomial k used to reduce F,G
#(See the end of Appendix A in "NTRUSign: Digital Signatures using the NTRU Lattice")
def Compute_k(f,g,F,G,N):
    RRR=RealField(350);
    phi = (x^N+1);
    FB = Reverse(F,N);
    GB = Reverse(G,N);
    fb = Reverse(f,N);
    gb = Reverse(g,N);
    num = (fb*F+gb*G).quo_rem(phi)[1];
    den = (f*fb+g*gb).quo_rem(phi)[1];
    (a,iden,iphi) = xgcd(den,x^N+1);
    k0 = (num*iden).quo_rem(phi)[1];
    k = sum( (k0[i]//a[0])*x^i for i in [0..N-1]  );
    k = k.change_ring(ZZ);
    return k;


def Rotate(v,k):
    w = list(v);
    for i in range(k):
        w.insert(0,-w.pop());
    return vector(w);


#Returns the Anticirculant matrix A_N(f) generated by, f, x^k.f, ..., x^((N-1)k).f
def AC(f,N,k):
    u = f.coefficients();
    while(len(u)<N):
        u.append(0);
    A = matrix(ZZ,N);
    z = vector(u);
    for i in range(N):
        A[i] = z;
        z = Rotate(z,k);
    return A;


#Tests if f is invertible mod X^N+1 mod q
def Is_Invertible(f,N,q):
    (pgcd,u,v) = xgcd(f,x^N+1);
    rep = gcd(pgcd,q);
    return (rep==1);


#Computes the inverse of f mod X^N+1 mod q
def Inverse(f,N,q):
    (pgcd,u,v) = xgcd(f,x^N+1);
    p_1 = inverse_mod(pgcd[0],q);
    u = p_1*u;
    u = u.quo_rem(q)[1];
    return u;


#Computes h = g/f mod X^N+1 mod q
def h_from_fg(f,g,N,q):
    phi = x^N+1;
    f_1 = Inverse(f,N,q);
    h = ((f_1*g).quo_rem(phi)[1]).quo_rem(q)[1];
    return h;


#Returns the NTRU secret basis generated by f,g
def NTRU_Secret_Basis(f,g,N,q):
    (F,G) = Derivation_FG(f,g,N,q);
    #print (f*G - g*F == q)
    print (f)
    print (g)
    print (F)
    print (G)
    A = AC(f,N,1);
    B = AC(g,N,1);
    C = AC(F,N,1);
    D = AC(G,N,1);
    E = block_matrix([[A,B],[C,D]]);
    return E;


#Returns the NTRU public basis generated by f,g
def NTRU_Public_Basis(f,g,N,q):
    phi = x^N+1;
    h = h_from_fg(f,g,N,q);
    A = identity_matrix(ZZ,N);
    B = AC(h,N,1);
    C = zero_matrix(ZZ,N);
    D = q*identity_matrix(ZZ,N);
    E = block_matrix([[A,B],[C,D]]);
    return E;


#Push-button procedure for generating the public and private bases for a NTRU lattice
#The expected norms of f,g is hardcoded ('norm') but you can change it
def Keygen(N,q):
    norm = isqrt(q)//2;
    Rep = False;
    while(Rep==False):
        f = Rand_Pol_Fixed_Norm(N,norm);
        g = Rand_Pol_Fixed_Norm(N,norm);
        Rep = Test(f,g,N,q);
        if(Rep==True):
            Rep = Is_Invertible(f,N,q);
    Sk = NTRU_Secret_Basis(f,g,N,q);
    Pk = NTRU_Public_Basis(f,g,N,q);
    return (Sk,Pk);

Keygen(251, 128)

-x^248 - x^244 - x^242 - x^239 - x^237 - x^236 - x^231 - x^228 - x^220 - x^219 - x^218 - x^215 - x^214 - x^213 - x^212 - x^207 - x^206 - x^204 - x^203 - x^202 - x^201 - x^193 - x^192 - x^186 - x^185 - x^183 - x^181 - x^180 - x^178 - x^177 - x^174 - x^171 - x^170 - x^160 - x^156 - x^154 - x^152 - x^150 - x^148 - x^145 - x^143 - x^141 - x^140 - x^139 - x^137 - x^136 - x^130 - x^126 - x^122 - x^120 - x^119 - x^117 - x^114 - x^112 - x^111 - x^109 - x^107 - x^104 - x^103 - x^99 - x^97 - x^96 - x^95 - x^92 - x^91 - x^85 - x^81 - x^79 - x^78 - x^77 - x^76 - x^75 - x^74 - x^73 - x^71 - x^67 - x^65 - x^64 - x^61 - x^60 - x^58 - x^57 - x^56 - x^54 - x^52 - x^51 - x^50 - x^49 - x^47 - x^45 - x^42 - x^37 - x^34 - x^32 - x^30 - x^28 - x^25 - x^20 - x^19 - x^18 - x^16 - x^15 - x^14 - x^12 - x^11 - x^9 - x^8 - x^3 - x^2 - x - 1
-x^249 - x^243 - x^240 - x^239 - x^238 - x^233 - x^231 - x^230 - x^229 - x^228 - x^222 - x^221 - x^218 - x^217 - x^216 - x^213 - x^209 - x^208 - x^205 - x^203 - x^202 - x^199 

(502 x 502 dense matrix over Integer Ring,
 502 x 502 dense matrix over Integer Ring)